In [1]:
# coding=utf-8
import json
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict
import jieba
import jieba.posseg as pseg

引入知识库，资料从网络上收集并已经加以整理。

In [2]:
name_data = pd.read_excel("name_data.xlsx",index_col=0)
name_data = name_data.set_index("姓名")
name_data = name_data.fillna("")

In [3]:
name_data.head()

,姓,名,字
姓名,,,
丁仪,丁,仪,正礼
丁冲,丁,冲,无
丁原,丁,原,建阳
丁厷,丁,厷,无
丁君干,丁,君干,无


下面用过程式的方法演示字典树（实体匹配型）的构造过程。相比我实际应用中的有所简化，更详细的代码和数据可以在我的github上找到：

In [4]:
trie_root = {}
def build_trie(new_word,entity,trie_root):
    trie_node = trie_root
    for ch in new_word:
        if not ch in trie_node:
            trie_node[ch] = {}
        trie_node = trie_node[ch]
    if not 'leaf' in trie_node:
        trie_node['leaf'] = set([entity])    #同一个代称有时会对应多个实体，所以这里的叶结点是集合类型
    else:
        trie_node['leaf'].add(entity)

In [5]:
build_trie("玄德","刘备",trie_root)
build_trie("玄德公","刘备",trie_root)
build_trie("刘皇叔","刘备",trie_root)
build_trie("刘备","刘备",trie_root)

In [6]:
print(trie_root)

{'玄': {'德': {'leaf': {'刘备'}, '公': {'leaf': {'刘备'}}}}, '刘': {'皇': {'叔': {'leaf': {'刘备'}}}, '备': {'leaf': {'刘备'}}}}


在字典树中查找代称对应的实体。

In [7]:
def find_trie(word,trie_root):
    trie_node = trie_root
    for ch in word:
        if not ch in trie_node:
            return set()
        trie_node = trie_node[ch]
    if "leaf" in trie_node:
        return trie_node['leaf']
    else:
        return set()

In [8]:
print(find_trie("刘备",trie_root))
print(find_trie("刘备啊",trie_root))

{'刘备'}
set()


但这里我需要的其实是把字典树与顺序扫描结合在一起，一边扫描一边识别实体。这样其实我是不能准确得到词语的右边界的。于是我需要另一个函数来完成这个任务，逐字符扫描中深入字典树【每一步暂存当前结点】，直到发现实体（集）或者无法匹配再跳出。

In [9]:
def dig_trie(sent,l,trie_root):                                #返回实体右边界r,实体范围
    trie_node = trie_root
    for i in range(l,len(sent)):
        if sent[i] in trie_node:
            trie_node = trie_node[sent[i]]
        else:
            if "leaf" in trie_node:
                return i, trie_node["leaf"]
            else:
                return -1, set()                                 # -1表示未找到
    # 收尾
    if "leaf" in trie_node:
        return len(sent), trie_node["leaf"]
    else:
        return -1, set()                                 # -1表示未找到

In [10]:
build_trie("刘","666",trie_root)
build_trie("刘胜","刘胜",trie_root)

In [11]:
print(dig_trie("中山靖王刘胜之后，汉景帝阁下玄孙",0,trie_root))
print(dig_trie("中山靖王刘胜之后，汉景帝阁下玄孙",4,trie_root))          # 不会被单独的刘干扰，除非确实不匹配
print(dig_trie("中山靖王刘某之后，汉景帝阁下玄孙",4,trie_root))

(-1, set())
(6, {'刘胜'})
(5, {'666'})


In [13]:
def entity_linking(sent,trie_root):
    ret = []
    l = 0
    while l < len(sent):
        r, entities = dig_trie(sent,l,trie_root)
        if r != -1:
            name0 = sent[l:r]
            ret.append(([l,r],list(entities)[0])) #简单起见，在这里如果同一个名称对应多个实体，默认选取第一个，完整版中用了更多消歧方法
            l = r
        else:
            l += 1
    return ret
sent = "刘备字玄德，号称刘皇叔，是刘氏又一豪杰"
print(entity_linking(sent,trie_root))

[([0, 2], '刘备'), ([3, 5], '刘备'), ([8, 11], '刘备'), ([13, 14], '666')]


如果只是要统计实体的出现次数的话，到这里已经足够，如果再想利用这些知识进行分词的话，还有几步要做。

首先，利用我们所有的姓名知识库构建一颗完整的字典树。

In [14]:
names_trie = {}
for entity0, line in name_data.iterrows():
    for name0 in [line["姓"]+line["名"],line["字"]]:               
        if not name0 in ["无","None",""] and len(name0) > 1:            
            build_trie(name0,entity0,names_trie)
    name0  = line["姓"]+line["名"]
    if not name0 in ["无","None",""] and len(name0) > 1:            # 长度1的名字太容易混淆，不采纳
        build_trie(name0,entity0,names_trie)

In [15]:
names_trie["子"]

{'上': {'leaf': {'司马昭', '纪陟'}},
 '业': {'leaf': {'卫继'}},
 '丹': {'leaf': {'曹真'}},
 '举': {'leaf': {'荀恺'}},
 '义': {'leaf': {'太史慈'}},
 '乔': {'leaf': {'孙松', '张松[刘璋]'}},
 '云': {'leaf': {'卢浮', '张津', '杨雄'}},
 '产': {'leaf': {'张特'}},
 '京': {'leaf': {'魏讽'}},
 '仁': {'leaf': {'刘惇', '毌丘宗'}},
 '仲': {'leaf': {'吴康', '虞放', '麋竺'}},
 '休': {'leaf': {'暨艳'}},
 '伯': {'leaf': {'娄圭'}},
 '佩': {'leaf': {'韩珩'}},
 '修': {'leaf': {'上官雝', '曹昂'}},
 '元': {'leaf': {'司马师', '濮阳兴'}},
 '全': {'leaf': {'王双[讨蜀]'}},
 '公': {'leaf': {'吴霸'}},
 '冥': {'leaf': {'子冥'}},
 '初': {'leaf': {'刘巴[蜀]', '司马望'}},
 '华': {'leaf': {'司马芝'}},
 '卿': {'leaf': {'严武'}},
 '叔': {'leaf': {'邯郸淳'}},
 '台': {'leaf': {'刘勋[魏]', '张阁'}},
 '叹': {'leaf': {'顾徽'}},
 '和': {'leaf': {'徵崇', '曹纯'}},
 '均': {'leaf': {'王平'}},
 '坚': {'leaf': {'李固[东汉大臣]'}},
 '太': {'leaf': {'郝普'}},
 '奇': {'leaf': {'刘陶[东汉]'}},
 '威': {'leaf': {'孙霸'}},
 '婴': {'leaf': {'子婴'}},
 '孝': {'leaf': {'孙和', '曹仁'}},
 '孟': {'leaf': {'霍光'}},
 '安': {'leaf': {'曹峻'}},
 '宪': {'leaf': {'李孚'}},
 '家': {'leaf': {'卢毓'}},
 

关键的一个小操作，指定一个标准词与所有实体联系起来，保证分词工具能够把它分割出来，并且赋予恰当的词性("nr"：人名)。

In [16]:
jieba.add_word("人占位符",freq=10000,tag="nr")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\KELEN\AppData\Local\Temp\jieba.cache
Loading model cost 0.927 seconds.
Prefix dict has been built succesfully.


In [17]:
def decoref(sent,entities_info):
    left = 0
    processed_text = ""
    for (beg,end),entity in entities_info:
        print(sent[beg:end],entity)
        processed_text += sent[left:beg] + "人占位符"
        left = end
    processed_text += sent[left:]
    return processed_text

In [18]:
sent = "玄德幼孤，事母至孝；家贫，贩屦织席为业。"
entities_info = [([0, 2], '刘备')]
print(decoref(sent,entities_info))

玄德 刘备
人占位符幼孤，事母至孝；家贫，贩屦织席为业。


看到这里我们的标准词"人占位符"已经替代了识别到的实体的位置，分词时，我们则会把它再换回来，并且提取出了我们设定的词性。

In [19]:
def posseg(sent,trie_root):
    entities_info = entity_linking(sent,trie_root)
    sent2 = decoref(sent,entities_info)
    result = []
    i = 0
    for word, flag in pseg.cut(sent2):
        if word == "人占位符":
            word = entities_info[i][1]
            i += 1
        result.append((word, flag))
    return result

In [20]:
print(posseg("玄德幼孤，事母至孝；家贫，贩屦织席为业。",names_trie))

玄德 刘备
[('刘备', 'nr'), ('幼', 'ag'), ('孤', 'ng'), ('，', 'x'), ('事母至孝', 'l'), ('；', 'x'), ('家贫', 'b'), ('，', 'x'), ('贩', 'v'), ('屦', 'g'), ('织', 'v'), ('席为业', 'nr'), ('。', 'x')]


这次第一个“刘备”君被成功地识别了出来。不过还有一个有趣的插曲是，分词工具“识别”出了另一个人名“席为业”。这个问题也可以用我们的知识库解决，毕竟统计的时候只需要统计我们知识库中已经出现的人名就好了吗。

最后附上一些我基于这个数据做的一些简单分析。

将出现在两句话以内的每一对人物添加一条边，统计了整个三国演义的文本后，我们就能够得到一个三国演义的人物网络。